In [37]:
import pandas as pd
import numpy as np

from gait_pipeline import topological_transform

from pre_process import find_period_acf
from construct_dataset import load_data_from_json


filename = './Datasets/left_thigh_1.json'

In [38]:
full_dataset = load_data_from_json(filename)

type(full_dataset[1])

list

In [39]:
entropies = []
for subject in full_dataset.keys():
    for measurement in full_dataset[subject]:
        measurement = np.array(measurement['acc_lt_z']).reshape(1,-1)
        H0, H1 = topological_transform(measurement)[0]
        entropies.append([subject, H0, H1])

In [40]:
entropy_df = pd.DataFrame(entropies, columns = ['subject', 'H0_Entropy', 'H1_Entropy'])

In [41]:
from sklearn.model_selection import train_test_split

X = entropy_df.drop('subject', axis = 1)
y = entropy_df['subject']

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=.15)

In [42]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=6)

In [43]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6)

In [44]:
y_pred = clf.predict(X_test)

In [45]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.43      0.43      0.43         7
           3       0.25      0.67      0.36         6
           4       0.25      0.29      0.27         7
           5       1.00      0.60      0.75         5
           6       1.00      0.11      0.20         9
           7       0.00      0.00      0.00         8
           8       0.60      0.60      0.60         5
           9       0.33      0.29      0.31         7
          10       0.50      0.09      0.15        11
          11       0.17      0.08      0.11        12
          12       0.08      0.50      0.14         6
          14       0.50      0.20      0.29         5
          15       0.38      0.50      0.43         6
          16       0.50      0.38      0.43         8
          17       0.31      0.44      0.36         9
          18       0.00      0.00      0.00         9

    accuracy                           0.28       120
   macro avg       0.39   

/home/dan-bossaller/Desktop/TDA/HuGaDB: Human Gait Database/hugait_tda/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dan-bossaller/Desktop/TDA/HuGaDB: Human Gait Database/hugait_tda/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dan-bossaller/Desktop/TDA/HuGaDB: Human Gait Database/hugait_tda/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with